In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

# carga prepro
df = pd.read_csv('/content/Indicadores_municipales_sabana_DA.csv', encoding='ISO-8859-1')
df_procesado = df.drop(columns=['nom_mun', 'nom_ent'])

# valores promedio de categorias
categorias = ['Muy bajo', 'Bajo', 'Medio', 'Alto', 'Muy alto']
for columna in df_procesado.columns:
    if columna.startswith('gdo_rezsoc'):
        valores_promedio = {}
        for categoria in categorias:
            valores_promedio[categoria] = np.nanmean(df_procesado[df_procesado[columna] == categoria].select_dtypes(include=[np.number]))

        for categoria in categorias:
            df_procesado[columna] = df_procesado[columna].replace(categoria, valores_promedio[categoria])

# rellenamos los valores faltantes con los promedios
promedio_global = np.nanmean(df_procesado.select_dtypes(include=[np.number]))
df_procesado = df_procesado.fillna(promedio_global)

# define df  eliminando la columna a predecir
#matriz caracteristicas x y la variable objetivo y
X = df_procesado.drop(columns=['N_carencias'])
y = df_procesado['N_carencias']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# asume los datos se distribuyen de acuerdo a dis gaussiana///clasificacion
#se puede adaptar con partial fit
modelo_naive_bayes_regresion = GaussianNB()

# utilizamos partial fit para ajustar el modelo incrementalmente
modelo_naive_bayes_regresion.partial_fit(X_train, y_train, np.unique(y))

# partial.fit permite entrenar el modelo de lotes sucesivos
#np.unique especifica las clases unicas presentes en el conjunto de etiquetas y
#realiza predicciones, se almacena en y pred
y_pred_naive_bayes_regresion = modelo_naive_bayes_regresion.predict(X_test)

#
precision_naive_bayes_regresion = metrics.r2_score(y_test, y_pred_naive_bayes_regresion)
print(f"Precisión del modelo Naive Bayes para regresión: {precision_naive_bayes_regresion}")

#muestra las ultimas diez reales y predictivas
ultimas_diez_real = df['N_carencias'].iloc[-10:]
indices_ultimas_diez = ultimas_diez_real.index

#elimina y adecua
X_ultimas_diez = df_procesado.loc[indices_ultimas_diez].drop(columns=['N_carencias'])
y_pred_naive_bayes_regresion_ultimas_diez = modelo_naive_bayes_regresion.predict(X_ultimas_diez)

# muestra
df_resultados_ultimas_diez = pd.DataFrame({
    'Real (N_carencias)': ultimas_diez_real.values,
    'Naive Bayes Regresión': y_pred_naive_bayes_regresion_ultimas_diez
}, index=indices_ultimas_diez)

print(df_resultados_ultimas_diez)

/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:514: RuntimeWarning: divide by zero encountered in log
  jointi = np.log(self.class_prior_[i])


Precisión del modelo Naive Bayes para regresión: -0.0730255841815397
      Real (N_carencias)  Naive Bayes Regresión
2446               30578                  14262
2447               10693                  10879
2448               38751                  38751
2449               17558                  17558
2450                9766                   9766
2451               19772                  14262
2452               24393                  24393
2453               54282                   6130
2454               18845                  14262
2455                2486                   2486


/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:514: RuntimeWarning: divide by zero encountered in log
  jointi = np.log(self.class_prior_[i])
